<a href="https://colab.research.google.com/github/ShibleNK/pytorch-practice/blob/main/building_neural_nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [7]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [8]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
          )
  def forward(self, x):
        x = self.flatten(x)
        logits =self.linear_relu_stack(x)
        return logits


In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [16]:
layer1 = nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0216,
         0.2020, 0.0946, 0.0225, 0.0000, 0.0000, 0.0000, 0.5969, 0.0000, 0.0770,
         0.1562, 0.4025],
        [0.0000, 0.0000, 0.0097, 0.0000, 0.0000, 0.1866, 0.0124, 0.0000, 0.0000,
         0.0000, 0.3068, 0.2804, 0.0000, 0.0000, 0.0000, 0.4785, 0.0000, 0.5815,
         0.3409, 0.4035],
        [0.0000, 0.0281, 0.2161, 0.0000, 0.0000, 0.4658, 0.0259, 0.0000, 0.0000,
         0.2973, 0.2612, 0.3668, 0.0000, 0.0000, 0.0000, 0.2013, 0.0000, 0.1286,
         0.1835, 0.1715]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0216,
         0.2020, 0.0946, 0.0225, 0.0000, 0.0000, 0.0000, 0.5969, 0.0000, 0.0770,
         0.1562, 0.4025],
        [0.0000, 0.0000, 0.0097, 0.0000, 0.0000, 0.1866, 0.0124, 0.0000, 0.0000,
         0.0000, 0.3068, 0.2804, 0.0000, 0.0000, 0.0000, 0.4785, 0.0000, 0.5815,
         0.3409, 0.4035],
       

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,

    nn.ReLU(),
    nn.Linear(20,10))
input_image = torch.rand(3, 28, 28)
logits =seq_modules(input_image)

In [21]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [22]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()}| Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784])| Values: tensor([[ 0.0227,  0.0181, -0.0302,  ..., -0.0142,  0.0260, -0.0261],
        [-0.0338, -0.0237,  0.0065,  ...,  0.0056, -0.0048, -0.0152]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512])| Values: tensor([ 0.0193, -0.0225], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512])| Values: tensor([[ 0.0440, -0.0369,  0.0018,  ...,  0.0301,  0.0366,  0.0346],
        [-0.0349,  0.0152,  0.0077,  ..., -0.0403,  0.0255,  0.0401]],
       device='cuda:0', grad_fn=<SliceBac